In [ ]:
# this notebook requires a scientific Python 3 install (numpy, scipy, matplotlib)
%pylab
import os, sys, shutil, PIL
save = True
open_image_windows = False
os.makedirs('fig', exist_ok=True)

# 2.1 simulated image quality: fig. 1

In [ ]:
output_folder = os.path.join('fig', 'effects')
try: 
    os.makedirs(output_folder)
except FileExistsError:
    #shutil.rmtree(output_folder)
    #os.makedirs(output_folder)
    pass
    
def bin_down_image(image: ndarray, scale_factor: int):
    if image.dtype.kind != 'f':
        image = image.astype("f4")
    if scale_factor == 1:
        return image
    else:
        downscaled = None
        stops = asarray(image.shape) - asarray(image.shape) % scale_factor
        for y_start in range(scale_factor):
            for x_start in range(scale_factor):
                if downscaled is None:
                    downscaled = copy(image[y_start:stops[0]:scale_factor, x_start:stops[1]:scale_factor])
                else:
                    downscaled += image[y_start:stops[0]:scale_factor, x_start:stops[1]:scale_factor]
        downscaled /= scale_factor**2
        return downscaled

def show_image(image, title=None, figsize=None, vrange=None, perc_val=(0.1, 99.9)):
    if figsize is None: figsize = 8, 8*image.shape[1]/image.shape[0]
    fig, ax = plt.subplots(1, 1, figsize=figsize); ax.axis('off')
    if vrange is None: vrange = nanpercentile(image, perc_val)
    ax.imshow(image, cmap=plt.cm.gray, interpolation='none', vmin=vrange[0], vmax=vrange[1])
    if title is not None: ax.set_title(title)
    if not open_image_windows:
        plt.close(fig)
    return fig

def save_image(image, filename, dtype=None, png_compress_level=4, vrange=(0, 255)):
    if save:
        if dtype is not None: image = image.astype(dtype)
        if filename.endswith('.raw') or '.' not in filename:
            image.tofile(filename)
        else:
            pil_args = dict()
            if filename[-3:] == 'png': 
                pil_args['compress_level'] = png_compress_level
                image = ((image.clip(vrange[0], vrange[1])-vrange[0])*(255/(vrange[1]-vrange[0]))).astype('u1')
            PIL.Image.fromarray(image).save(filename, **pil_args)
        #print(f'saved image {filename}')
    
def save_both(image, fname, vrange=None, perc_val=(0.1, 99.9)):
    display(show_image(image, title=os.path.split(fname)[1]))
    if save:
        save_image(image, fname+'.tif')
        if vrange is None: vrange = nanpercentile(image, perc_val)
        save_scale(fname, vrange)
        save_image(image, fname+'.png', vrange=vrange)
    
def save_scale(fname, vrange):
    folder, file_name = os.path.split(fname)
    with open(os.path.join(folder, 'scales.txt'), 'a') as fp:
        fp.write(f'{file_name}\t{vrange[0]:.3f}\t{vrange[1]:.3f}\n')

## simulated signal

The simulated image is a projection of balls of different diameters with the absorption strengths inversely proportional to the ball diameter.

The placement is randomized because in a real case, the placement would not be known. This avoids only detecting a ball because the position is known beforehand.

In [ ]:
def generate_balls_pattern(return_all=False):
    size = array((100, 200))
    oversampling = 10
    size_os = size*oversampling
    image = ones(size_os, 'f4')
    
    radii = logspace(log10(0.75), log10(3), 5)[::-1]*oversampling
    nballs = zeros(len(radii), 'u1')
    space = (size_os[1] - 2*radii.sum())/(len(radii)+1)
    grid_pos_x = zeros(radii.shape[0], 'i4')
    for k, radius in enumerate(radii):
        grid_pos_x[k] = (k+1)*space + radii[:k].sum() + radii[:k+1].sum()
    grid_pos_x = (grid_pos_x/oversampling).astype('i4')*oversampling
    
    coord_y_base = arange(image.shape[0])
    coord_x_base = arange(image.shape[1])
    for k, pos_x in enumerate(grid_pos_x):
        radius = radii[k]
        grid_pos_y = linspace(0, size_os[0], int(0.15*size_os[0]/(radius+7))+2, endpoint=True)[1:-1].astype('i4')
        nballs[k] = len(grid_pos_y)
        max_xshift = space/6/oversampling
        max_yshift = 1.5*radius/oversampling
        for pos_y in grid_pos_y:
            coord_x = coord_x_base - pos_x + random.uniform(-max_xshift, max_xshift)*oversampling
            coord_y = coord_y_base - pos_y + random.uniform(-max_yshift, max_yshift)*oversampling
            
            coord_r = sqrt(coord_x[newaxis, :]**2 + coord_y[:, newaxis]**2)
            locations = less(coord_r, radius)
            
            image[locations] *= exp(-sqrt(1 - (coord_r[locations]/radius)**2)*0.4)
        
    image = bin_down_image(image, oversampling)
        
    if return_all:
        return image, radii/oversampling, nballs
    else:
        return image
    
image, radii, nballs = generate_balls_pattern(True)
print('circle diameters:', *[f'{2*r:.2f}' for r in radii])
print('number of balls:', *[f'{k}' for k in nballs])

if save:
    savetxt('fig/effects/radii.txt', radii)
    savetxt('fig/effects/nballs.txt', nballs)
vrange = (0.5, 1.3)
vrange = None
save_both(image, os.path.join(output_folder, 'signal'), vrange=vrange)

# (A) noisiness

In [ ]:
countss = 3000, 1000, 300, 100
for counts in countss:
    image = generate_balls_pattern()
    image_c = image*counts
    image_sampled = random.poisson(image_c)
    image_sampled = image_sampled / counts
    save_both(image_sampled, os.path.join(output_folder, f'noisiness_c={counts}'), vrange=vrange)

# (B) modulation transfer

In [ ]:
from scipy.ndimage import gaussian_filter
counts = 1000
blurinesss = (0.5, 1., 2.0, 4.0)
for bluriness in blurinesss:
    image = generate_balls_pattern()
    image_c = image*counts
    image_c = gaussian_filter(image_c, bluriness)
    image_sampled = random.poisson(image_c)
    image_sampled = image_sampled / counts
    save_both(image_sampled, os.path.join(output_folder, f'modulation_sigma={bluriness:.1f}'.replace('.', '_')), vrange=vrange)

# (C) signal strength

In [ ]:
counts = 2500
bluriness = 0.2
signal_strengths = (1, 0.5, 0.25, 0.125)
for signal_strength in signal_strengths:
    image = generate_balls_pattern()
    image_c = exp(log(image)*signal_strength)
    
    image_c = image_c*counts
        
    image_sampled = random.poisson(image_c)
    image_sampled = image_sampled / counts
    save_both(image_sampled, os.path.join(output_folder, f'signal_strength={signal_strength:.2f}'.replace('.', '_')), vrange=vrange)

# (D) superposition

In [ ]:
def simulate_n():
    image = generate_balls_pattern()
    
    counts_1 = 1000
    image_c = exp(log(image)*1)
    image_c = image_c*counts_1
    image_1_sampled = random.poisson(image_c)

    counts_2 = 10000
    image_c = exp(log(image)*0.1)
    image_c = image_c*counts_2
    image_2_sampled = random.poisson(image_c)

    image_12_sampled = (image_1_sampled+image_2_sampled)/(counts_1 + counts_2)
    image_1_sampled = image_1_sampled / counts_1
    image_2_sampled = image_2_sampled / counts_2
    return image_1_sampled, image_2_sampled, image_12_sampled

def simulate_b():
    image = generate_balls_pattern()
    
    counts_1 = 1000
    image_c = image*counts_1
    image_1_sampled = random.poisson(image_c)

    counts_2 = 3000
    image_c = gaussian_filter(image, 2.0)
    image_c = image_c*counts_2
    image_2_sampled = random.poisson(image_c)

    image_12_sampled = (image_1_sampled+image_2_sampled)/(counts_1 + counts_2)
    image_1_sampled = image_1_sampled / counts_1
    image_2_sampled = image_2_sampled / counts_2
    return image_1_sampled, image_2_sampled, image_12_sampled

image_1_sampled = simulate_n()[0]

image_2n_sampled = simulate_n()[1]

image_12n_sampled = simulate_n()[2]
    
image_2b_sampled = simulate_b()[1]

image_12b_sampled = simulate_b()[2]

save_both(image_1_sampled, os.path.join(output_folder, 'superposition_image_1'), vrange=vrange)
save_both(image_2n_sampled, os.path.join(output_folder, 'superposition_image_2_weak_signal'), vrange=vrange)
save_both(image_12n_sampled, os.path.join(output_folder, 'superposition_signal_strengths'), vrange=vrange)
save_both(image_2b_sampled, os.path.join(output_folder, 'superposition_image2_bad_MTF'), vrange=vrange)
save_both(image_12b_sampled, os.path.join(output_folder, 'superposition_MTF_qualities'), vrange=vrange)

In [ ]:
plt.close('all')